In [2]:
## base llm
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

from langgraph.graph import StateGraph, Graph, START, END
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import MemorySaver



class SophyState(TypedDict):
    exchange: int
    conv_sum: str
    conversation_history: list[AnyMessage]
    latest_exchanges: list[AnyMessage]
    user_input: str

In [3]:
ss_agent_key = ""

In [5]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=ss_agent_key
)

In [6]:
llm.invoke("hi")

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.108966189, 'prompt_time': 0.010023296, 'queue_time': 0.017561643000000002, 'total_time': 0.118989485}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c0cfa69934', 'finish_reason': 'stop', 'logprobs': None}, id='run-8370b83c-84ab-4619-b12e-fefd4afd793c-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [7]:
from pydantic import BaseModel, Field

class AgentResponse(BaseModel):
    "Route to proceed in"
    pickedRoute: str = Field(description= "The route to proceed in, can only be: 'continue_chat', 'crisis_hepline', 'resource_suggestion', 'end_chat' ")
    reason: str =Field(description ="Reason for picking the route")

In [8]:
select_conv = [HumanMessage(content='When they first said those things, I felt like I was everything to them. It made me feel like I had a place in their life, like we were partners in every sense of the word. I remember feeling this deep sense of connection and trust, like we were building something that couldn’t be broken. But now, looking back, those words just feel like a performance, something to keep me from seeing the truth. They weren’t saying them because they meant them—they were saying them to keep me attached while they did something completely different. Over time, the feeling of warmth that came with those words has turned into a sense of betrayal. It’s hard to even hear those phrases now without feeling angry or deceived. It’s like they were always just playing a role.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's as if you felt seen and understood by them, like your feelings and needs were being acknowledged. But now, it feels like they were using those words to maintain control over you, even though the actions didn't match the promises. The pain of feeling deceived is still there, but it's also accompanied by a sense of anger and hurt.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Exactly. It’s like they made me believe that I was their priority, that they saw me and cared for me, but it was all just part of some act. The worst part is that I gave them everything—my trust, my love, my vulnerability. I let my guard down completely, and they used that to keep me close while they were doing something entirely different behind my back. It’s hard not to feel betrayed on so many levels.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's like you poured your heart out to them, and they took advantage of that trust. The fact that you gave them everything – your love, vulnerability – makes it even more hurtful when they used that for their own gain. It's like they were playing a game with you, and you got played.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Yeah, exactly. I feel like I was just a piece in their game, something they could manipulate to get what they wanted, and it makes everything feel so cheap.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It sounds like you're feeling a sense of powerlessness and disconnection from the situation. You're recognizing that your feelings and needs were being used, even though the actions didn't match the promises. That can be really painful and isolating.", additional_kwargs={}, response_metadata={})]

In [9]:
user_input_end = "This is all i wanted to talk for today bye"
user_input_crisis = "I might want to kill myself"
user_input_resources = "can you suggest any resources?"
user_input_chat = "Yeah, you are right."

In [10]:
llm_agent_response = llm.with_structured_output(AgentResponse)

In [12]:
def get_prompt1(select_conv, ui):
    prompt = f'''
        You are an autonoums agent in a mental health support chat system.

        Based on the current human input and previous exchanges in the conversation pick the best route for the conversation to proceed in.
        
        Previous Exchanges:
        {select_conv}
        
        Current Human Input:
        {ui}

        Routes:
        continue_chat : Normal chat route with mental health assistant
        crisis_hepline : Speaclized route connects user to 24X7 crisis helpline to professionals will help the human avoid crisis like suicide and other harms.
        resource_suggestion : Specialized provider of resources suggestions for coping techniques, therpaists contacts
        end_chat : Specialized end message if the user intends to end the chat

        If you can not provide answers for sucidial tendencies or harmful pick the crisis helpine route so that user can recieve help from professionals.

        If undecided and there are no harmful intentions always pick continue_chat

        '''
    
    return prompt

def get_prompt2(ui):

    prompt2 = f'''
        You are an autonoums agent in a mental health support chat system.

        Based on the current human input of a conversation with mental health assistant pick the best route for the conversation to proceed in.
        
        Current Human Input:
        {user_input_crisis}

        Routes:
        continue_chat : Normal chat route with mental health assistant
        crisis_hepline : Speaclized route connects user to 24X7 crisis helpline to professionals will help the human avoid crisis like suicide and other harms.
        resource_suggestion : Specialized provider of resources suggestions for coping techniques, therpaists contacts
        end_chat : Specialized end message if the user intends to end the chat

        If you can not provide answers for sucidial tendencies or harmful pick the crisis helpine route so that user can recieve help from professionals.

        If undecided and there are no harmful intentions always pick continue_chat

        '''
    
    return prompt2
# response = llm.invoke(prompt)

In [13]:

response_a = llm_agent_response.invoke(get_prompt1(select_conv, ui=user_input_chat))
response_a

AgentResponse(pickedRoute='continue_chat', reason="The user is acknowledging the understanding of their situation and no harmful intentions are expressed, so it's best to continue the conversation to provide further support and guidance.")

In [14]:

response_a = llm_agent_response.invoke(get_prompt1(select_conv, ui=user_input_crisis))
response_a

AgentResponse(pickedRoute='crisis_hepline', reason='The user has expressed suicidal tendencies, which requires immediate attention and support from professionals.')

In [15]:
response_a = llm_agent_response.invoke(get_prompt1(select_conv, ui=user_input_resources))
response_a

AgentResponse(pickedRoute='resource_suggestion', reason='The user has explicitly asked for resources, indicating a desire for support and coping techniques, which is best addressed by the resource_suggestion route.')

In [16]:
response_a = llm_agent_response.invoke(get_prompt1(select_conv, ui=user_input_end))
response_a

AgentResponse(pickedRoute='end_chat', reason='The user has explicitly stated that they want to end the chat for today, indicating a desire to conclude the conversation.')

In [17]:
select_conv = [HumanMessage("Can you suggest some resources?"), AIMessage("Here are some resources: link to meditation techniques, link to coping techniques")]
user_input_lazy = "Man I don't want to go through these"

response_a = llm_agent_response.invoke(get_prompt1(select_conv, ui=user_input_lazy))
response_a

AgentResponse(pickedRoute='continue_chat', reason='The user is expressing frustration with the provided resources, but there is no indication of suicidal tendencies or harmful intentions, so we will continue the chat to better understand their needs and provide further support.')

In [ ]:
response_a["pickedRoute"]